In [1]:
import torch.nn as nn
import torchvision.models as models

class ResnetV1(nn.Module):
    def __init__(self, num_classes):
        super(ResnetV1, self).__init__()
        # Load the pre-trained ResNet-50 model
        self.resnet = models.resnet50(pretrained=True)
        
        # Freeze all layers
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Unfreeze the last 4 parameters (typically layers close to the output)
        for param in list(self.resnet.parameters())[-4:]:
            param.requires_grad = True
        
        # Modify the last fully connected layer to match the number of classes
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x):
        x = self.resnet(x)
        return x


In [2]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

class Trainer:
    def __init__(
        self,
        model,
        data_loader_treino,
        data_loader_validacao,
        num_imagens_treino,
        num_imagens_validacao,
        device,
        num_classes=8,
        patience=5,
        nameModel='nome do arquivo.pt',
        otimizador=None,
        scheduler=None
    ):
        """
        Inicializa o Trainer com opções para usar ou não otimizador, scheduler e patience.

        Parâmetros:
        - model: O modelo a ser treinado.
        - data_loader_treino: DataLoader para o conjunto de treino.
        - data_loader_validacao: DataLoader para o conjunto de validação.
        - num_imagens_treino: Número total de imagens de treino.
        - num_imagens_validacao: Número total de imagens de validação.
        - device: Dispositivo onde o modelo será treinado (CPU ou GPU).
        - num_classes: Número de classes de saída.
        - patience: Número de épocas para o early stopping (opcional, pode ser None).
        - nameModel: Nome do arquivo para salvar o melhor modelo.
        - otimizador: O otimizador a ser usado (opcional, pode ser None).
        - scheduler: O scheduler de learning rate a ser usado (opcional, pode ser None).
        """
        self.model = model
        self.data_loader_treino = data_loader_treino
        self.data_loader_validacao = data_loader_validacao
        self.num_imagens_treino = num_imagens_treino
        self.num_imagens_validacao = num_imagens_validacao
        self.device = device
        self.funcao_erro = nn.CrossEntropyLoss()
        
        # Inicializa o otimizador apenas se for fornecido, senão cria um padrão
        if otimizador is None:
            self.otimizador = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-4)
        else:
            self.otimizador = otimizador
        
        # Inicializa o scheduler apenas se for fornecido
        self.scheduler = scheduler
        
        # Inicializa o patience
        self.patience = patience

        # Atualiza o caminho para salvar o modelo na pasta 'models/'
        self.nameModel = nameModel
        self.model_save_path = os.path.join('models', self.nameModel)
        os.makedirs(os.path.dirname(self.model_save_path), exist_ok=True)

    def treinar_e_validar(self, epocas):
        historico = []
        melhor_acuracia = 0.0
        early_stop_counter = 0

        for epoca in range(epocas):
            inicio_epoca = time.time()
            print(f"\n\nÉpoca: {epoca + 1}/{epocas}")
            erro_treino, acuracia_treino = self.executar_fase('treino')
            erro_validacao, acuracia_validacao, predicoes_validacao, labels_validacao = self.executar_fase('validacao', return_predictions=True)

            fim_epoca = time.time()
            print(f"Época {epoca + 1}/{epocas}, Treino: Erro: {erro_treino:.4f}, Acurácia: {acuracia_treino * 100:.2f}%, "
                  f"Validação: Erro: {erro_validacao:.4f}, Acurácia: {acuracia_validacao * 100:.2f}%, Tempo: {fim_epoca - inicio_epoca:.2f}s")

            historico.append([erro_treino, erro_validacao, acuracia_treino, acuracia_validacao])
            
            # Atualiza o scheduler se ele estiver definido e o scheduler não for None
            if self.scheduler is not None:
                self.scheduler.step(erro_validacao)

            # Early stopping
            if acuracia_validacao > melhor_acuracia:
                melhor_acuracia = acuracia_validacao
                print(f"Validation accuracy improved to {melhor_acuracia:.4f}. Saving the model.")
                try:
                    torch.save(self.model.state_dict(), self.model_save_path)
                    print(f"Modelo salvo com sucesso em {self.model_save_path}")
                except Exception as e:
                    print(f"Erro ao salvar o modelo: {e}")
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if self.patience is not None and early_stop_counter >= self.patience:
                print("Parando o treinamento devido ao early stopping.")
                break

        # Calcular métricas finais
        self.calcular_metricas(predicoes_validacao, labels_validacao)
        return historico

    def treinar_e_validar(self, epocas):
        historico = []
        melhor_acuracia = 0.0
        early_stop_counter = 0

        for epoca in range(epocas):
            inicio_epoca = time.time()
            print(f"\n\nÉpoca: {epoca + 1}/{epocas}")
            erro_treino, acuracia_treino = self.executar_fase('treino')
            erro_validacao, acuracia_validacao, predicoes_validacao, labels_validacao = self.executar_fase('validacao', return_predictions=True)

            fim_epoca = time.time()
            print(f"Época {epoca + 1}/{epocas}, Treino: Erro: {erro_treino:.4f}, Acurácia: {acuracia_treino * 100:.2f}%, "
                  f"Validação: Erro: {erro_validacao:.4f}, Acurácia: {acuracia_validacao * 100:.2f}%, Tempo: {fim_epoca - inicio_epoca:.2f}s")

            historico.append([erro_treino, erro_validacao, acuracia_treino, acuracia_validacao])
            
            # Atualiza o scheduler se ele estiver definido
            if self.scheduler is not None:
                self.scheduler.step(erro_validacao)

            # Early stopping
            if acuracia_validacao > melhor_acuracia:
                melhor_acuracia = acuracia_validacao
                print(f"Validation accuracy improved to {melhor_acuracia:.4f}. Saving the model.")
                try:
                    torch.save(self.model.state_dict(), self.model_save_path)
                    print(f"Modelo salvo com sucesso em {self.model_save_path}")
                except Exception as e:
                    print(f"Erro ao salvar o modelo: {e}")
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if self.patience is not None and early_stop_counter >= self.patience:
                print("Parando o treinamento devido ao early stopping.")
                break

        # Calcular métricas finais
        self.calcular_metricas(predicoes_validacao, labels_validacao)
        return historico

    def executar_fase(self, fase, return_predictions=False):
        if fase == 'treino':
            self.model.train()
            data_loader = self.data_loader_treino
            num_imagens = self.num_imagens_treino
        else:
            self.model.eval()
            data_loader = self.data_loader_validacao
            num_imagens = self.num_imagens_validacao

        erro_total = 0.0
        acuracia_total = 0.0
        todas_predicoes = []
        todas_labels = []

        with torch.set_grad_enabled(fase == 'treino'):
            print(f"\nExecutando a fase de {fase}...")
            for entradas, labels in data_loader:
                entradas, labels = entradas.to(self.device), labels.to(self.device)

                if fase == 'treino':
                    self.otimizador.zero_grad()

                saidas = self.model(entradas)
                erro = self.funcao_erro(saidas, labels)

                if fase == 'treino':
                    erro.backward()
                    self.otimizador.step()
                erro_total += erro.item() * entradas.size(0)
                _, predicoes = torch.max(saidas, 1)
                acuracia_total += (predicoes == labels).sum().item()

                if return_predictions:
                    todas_predicoes.extend(predicoes.cpu().numpy())
                    todas_labels.extend(labels.cpu().numpy())
                

        erro_medio = erro_total / num_imagens
        acuracia_media = acuracia_total / num_imagens

        if return_predictions:
            return erro_medio, acuracia_media, todas_predicoes, todas_labels
        else:
            return erro_medio, acuracia_media

    def calcular_metricas(self, predicoes, labels):
        acuracia = accuracy_score(labels, predicoes)
        precisao = precision_score(labels, predicoes, average='weighted', zero_division=0)
        recall = recall_score(labels, predicoes, average='weighted', zero_division=0)
        f1 = f1_score(labels, predicoes, average='weighted', zero_division=0)

        print("\nMétricas de Validação:")
        print(f"Acurácia: {acuracia:.4f}")
        print(f"Precisão: {precisao:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")

        print("\nRelatório de Classificação:")
        print(classification_report(labels, predicoes, zero_division=0))


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

class DataLoaderSetup:
    def __init__(self, dataset_path, image_size, batch_size, transformacoes=None):
        """
        Inicializa a classe DataLoaderSetup.

        Parâmetros:
        - dataset_path: Caminho para o dataset.
        - image_size: Tamanho da imagem para redimensionamento.
        - batch_size: Tamanho do batch.
        - transformacoes: Dicionário opcional com as transformações para 'treino' e 'validacao'. Se None, serão usadas transformações padrão.
        """
        self.dataset_path = dataset_path
        self.image_size = image_size
        self.batch_size = batch_size
        
        # Se não forem fornecidas transformações, usa as transformações padrão
        self.transformacoes = transformacoes or self.get_default_transforms()

    def get_default_transforms(self):
        """
        Define transformações padrão para o conjunto de treino e validação.
        """
        transformacoes_de_imagens = {
            'treino': transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.CenterCrop(self.image_size)
            ]),
            'validacao': transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.CenterCrop(self.image_size)
            ])
        }
        return transformacoes_de_imagens

    def get_data_loaders(self):
        """
        Cria DataLoaders para os conjuntos de treino e validação, aplicando as transformações fornecidas ou padrão.
        """
        # Usa as transformações fornecidas ou as padrão definidas no init
        train_dataset = datasets.ImageFolder(os.path.join(self.dataset_path, 'treino'), transform=self.transformacoes['treino'])
        val_dataset = datasets.ImageFolder(os.path.join(self.dataset_path, 'validacao'), transform=self.transformacoes['validacao'])

        data_loader_treino = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        data_loader_validacao = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)

        num_imagens_treino = len(train_dataset)
        num_imagens_validacao = len(val_dataset)
        num_classes = len(train_dataset.classes)

        return data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes


In [4]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = r'F:\Git\Teste\FER\affectnet\affectnet2'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1affectnet.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6279, Acurácia: 36.52%, Validação: Erro: 1.4662, Acurácia: 42.47%, Tempo: 296.75s
Validation accuracy improved to 0.4247. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnet.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.4395, Acurácia: 43.18%, Validação: Erro: 1.4230, Acurácia: 42.95%, Tempo: 172.44s
Validation accuracy improved to 0.4295. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnet.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.3954, Acurácia: 44.85%, Validação: Erro: 1.3679, Acurácia: 45.67%, Tempo: 172.72s
Validation accuracy improved to 0.4567. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnet.pt


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao...
Época 4

[[1.6278648461911254,
  1.4661666100749944,
  0.3652331137801886,
  0.4246905089408528],
 [1.4394513846541956,
  1.4229688770669362,
  0.4318309010288863,
  0.4295048143053645],
 [1.3953909757159981,
  1.367907644465996,
  0.4484911102501184,
  0.45667125171939477],
 [1.3707297045978544,
  1.353555865924001,
  0.46028671057729564,
  0.4604539202200825],
 [1.3570684513205518,
  1.3521042840844828,
  0.4639028800206638,
  0.4673314993122421],
 [1.349678012998245,
  1.3345038738342558,
  0.46842309182487407,
  0.47042640990371387],
 [1.331491158116243,
  1.3371683011520679,
  0.4731155021740066,
  0.4666437414030261],
 [1.3252812883857916,
  1.3197336688822205,
  0.4787119548839812,
  0.47455295735900965],
 [1.3198392546907183,
  1.3303926241938973,
  0.4787980541564424,
  0.47455295735900965],
 [1.3136997909550825,
  1.3135693266762336,
  0.4789702527013647,
  0.4769601100412655],
 [1.308870394131664,
  1.3099442687618355,
  0.48486805286495327,
  0.4783356258596974],
 [1.301774811583547

In [5]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/affectnet/processed/essenciais/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1affectnetessenciais.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6979, Acurácia: 36.31%, Validação: Erro: 1.5058, Acurácia: 42.64%, Tempo: 485.27s
Validation accuracy improved to 0.4264. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetessenciais.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.4960, Acurácia: 42.69%, Validação: Erro: 1.4275, Acurácia: 43.80%, Tempo: 158.75s
Validation accuracy improved to 0.4380. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetessenciais.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.4435, Acurácia: 43.95%, Validação: Erro: 1.4016, Acurácia: 45.08%, Tempo: 158.99s
Validation accuracy improved to 0.4508. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetessenciais.pt


Época: 4/30

Executando a fase de treino...

Executando 

[[1.6979372169696687,
  1.5058429845075658,
  0.36308368904410815,
  0.4264136328427576],
 [1.4960099654318377,
  1.4274964375979766,
  0.4268960462208425,
  0.4380325329202169],
 [1.4434710204336585,
  1.4016015018999437,
  0.43952916828353383,
  0.45081332300542215],
 [1.4189854374381317,
  1.3663937482235506,
  0.4457379381709999,
  0.4643687064291247],
 [1.4033259089477352,
  1.351819259357674,
  0.4522916397188807,
  0.4670797831138652],
 [1.392155813887116, 1.3680202941576887, 0.45530979174751, 0.4504260263361735],
 [1.3817006690389193,
  1.3476042109436992,
  0.45612900444099513,
  0.4736638264910922],
 [1.3624892836425466,
  1.325944576311444,
  0.4631569870219463,
  0.48644461657629745],
 [1.3572325073042841,
  1.3095541213668658,
  0.46919329107920493,
  0.4821843532145624],
 [1.348686320900028,
  1.318533822513199,
  0.469451989824516,
  0.47598760650658406],
 [1.348400058153401,
  1.304689733673675,
  0.4737205191221489,
  0.4790859798605732],
 [1.3426300237955233,
  1.3116

land

In [6]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/affectnet/processed/landmarks/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1affectnetlandmarks.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.7776, Acurácia: 31.33%, Validação: Erro: 1.7296, Acurácia: 33.46%, Tempo: 609.60s
Validation accuracy improved to 0.3346. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetlandmarks.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.6409, Acurácia: 36.62%, Validação: Erro: 1.6298, Acurácia: 35.54%, Tempo: 257.06s
Validation accuracy improved to 0.3554. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetlandmarks.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.5959, Acurácia: 37.99%, Validação: Erro: 1.5612, Acurácia: 39.88%, Tempo: 258.32s
Validation accuracy improved to 0.3988. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetlandmarks.pt


Época: 4/30

Executando a fase de treino...

Executando a f

[[1.7776170667395534,
  1.729570783975259,
  0.3132530120481928,
  0.3346139070303496],
 [1.6408512581110837,
  1.6298453097063168,
  0.36618815688285056,
  0.35535920092201306],
 [1.5959228699017638,
  1.5612443453341414,
  0.37994531316756386,
  0.39877064925086436],
 [1.5698288182850466,
  1.552867517258816,
  0.3900281978979749,
  0.3956972723780254],
 [1.5496451846251251,
  1.5131226936487614,
  0.3944287789455695,
  0.40645409143296196],
 [1.5265335188068447,
  1.4952008607430591,
  0.4060497308382466,
  0.41375336150595465],
 [1.517794475701694,
  1.50107334317769,
  0.40545159360847643,
  0.4091432961966961],
 [1.500019229905866,
  1.4646428098964361,
  0.4134837221225327,
  0.4310411064156742],
 [1.494526679145264,
  1.4962185981683076,
  0.4109202768520892,
  0.4172109104878986],
 [1.482153344967218,
  1.4707063357737538,
  0.41660258053490556,
  0.41413753361505956],
 [1.4797941949605269,
  1.4502768857269346,
  0.41873878492694183,
  0.42912024587014985],
 [1.47100123709861

In [7]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/affectnet/processed/landmarksConected/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1affectnetconected.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6344, Acurácia: 38.86%, Validação: Erro: 1.4777, Acurácia: 44.47%, Tempo: 683.89s
Validation accuracy improved to 0.4447. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetconected.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.4374, Acurácia: 44.81%, Validação: Erro: 1.3608, Acurácia: 48.89%, Tempo: 277.74s
Validation accuracy improved to 0.4889. Saving the model.
Modelo salvo com sucesso em models\modelo1affectnetconected.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.3948, Acurácia: 46.41%, Validação: Erro: 1.3262, Acurácia: 48.31%, Tempo: 277.70s


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao...
Época 4/30, Treino: Erro: 1.3620, Acurácia: 47.63%, Validação: Erro: 1.3133, Acurácia: 48.58%, Tempo: 27

[[1.6344390685494294,
  1.4777140070270411,
  0.3886351022499253,
  0.44474290099769764],
 [1.4374416699831092,
  1.3608037308373089,
  0.44806386884685995,
  0.488871834228703],
 [1.3948455501701018,
  1.3262386228703391,
  0.4641164667207446,
  0.48311588641596315],
 [1.3619992030612005,
  1.3132921475588315,
  0.47632668744396534,
  0.48580199539524177],
 [1.3443025905112975,
  1.2821239792010644,
  0.4801263715151774,
  0.5076745970836531],
 [1.3234139368932385,
  1.2624993929932506,
  0.4902446313452589,
  0.5126630851880276],
 [1.3143698144304108,
  1.2707752659059175,
  0.49109849293429536,
  0.5053722179585571],
 [1.3011801057879149,
  1.2439986480169816,
  0.4944712462109892,
  0.5249424405218726],
 [1.2903218626981103,
  1.2476912850338224,
  0.49745976177261664,
  0.522256331542594],
 [1.284608968493218,
  1.2277449664022955,
  0.5015156043205397,
  0.5303146584804298],
 [1.2710107656790028,
  1.2232449060207318,
  0.5092430517013192,
  0.5364543361473523],
 [1.2697150055749

fer

In [8]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/Fer-2013/processed/imagens_processed/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1fer.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6453, Acurácia: 33.13%, Validação: Erro: 1.6543, Acurácia: 32.44%, Tempo: 415.39s
Validation accuracy improved to 0.3244. Saving the model.
Modelo salvo com sucesso em models\modelo1fer.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.5504, Acurácia: 38.54%, Validação: Erro: 1.5298, Acurácia: 39.67%, Tempo: 142.51s
Validation accuracy improved to 0.3967. Saving the model.
Modelo salvo com sucesso em models\modelo1fer.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.5168, Acurácia: 40.35%, Validação: Erro: 1.5083, Acurácia: 41.10%, Tempo: 142.71s
Validation accuracy improved to 0.4110. Saving the model.
Modelo salvo com sucesso em models\modelo1fer.pt


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao...
Época 4/30, Treino: Erro:

[[1.6453449460667233,
  1.6542643858478823,
  0.33134030891878424,
  0.32438284971849285],
 [1.550409585549336,
  1.5298496054694373,
  0.3854287770580745,
  0.39670853183196186],
 [1.5168268419952744,
  1.5082545237088936,
  0.4035320821568953,
  0.41100043308791684],
 [1.4959442885500287,
  1.4994489785756524,
  0.4105630293971101,
  0.4140320485058467],
 [1.4896228000293148,
  1.4948832527183773,
  0.41078447655428224,
  0.4101342572542226],
 [1.4820344063858584,
  1.486858376593443,
  0.41465980180479434,
  0.4200952793417064],
 [1.4701800500344102,
  1.468585305009481,
  0.4247910092454188,
  0.4270246860112603],
 [1.464871777870697,
  1.4684663683679415,
  0.4260643303991585,
  0.42399307059333047],
 [1.4636305708270898,
  1.4736788549068016,
  0.4263965011349167,
  0.4174967518406236],
 [1.4493031711110587,
  1.466848348838109,
  0.4310468914355312,
  0.43265482893027285],
 [1.4463648591756701,
  1.4521092937951896,
  0.4312129768034103,
  0.43438718059766135],
 [1.4456200263682

In [9]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/Fer-2013/processed/essenciais/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1feressenciais.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6014, Acurácia: 36.22%, Validação: Erro: 1.4985, Acurácia: 41.07%, Tempo: 416.27s
Validation accuracy improved to 0.4107. Saving the model.
Modelo salvo com sucesso em models\modelo1feressenciais.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.5073, Acurácia: 40.88%, Validação: Erro: 1.4556, Acurácia: 43.20%, Tempo: 141.59s
Validation accuracy improved to 0.4320. Saving the model.
Modelo salvo com sucesso em models\modelo1feressenciais.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.4869, Acurácia: 41.70%, Validação: Erro: 1.4493, Acurácia: 41.81%, Tempo: 141.68s


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao...
Época 4/30, Treino: Erro: 1.4726, Acurácia: 42.25%, Validação: Erro: 1.4359, Acurácia: 43.07%, Tempo: 141.27s




[[1.6014212903103704,
  1.4984738498254706,
  0.36223218734429496,
  0.41074523396880414],
 [1.507347719248153,
  1.4555609457620922,
  0.40879145213973317,
  0.43197573656845756],
 [1.4869045999922217,
  1.4492980566140476,
  0.4169849969551016,
  0.41811091854419413],
 [1.4725598141857077,
  1.4358711494730085,
  0.4224658140951116,
  0.4306759098786828],
 [1.4562095842943275,
  1.4417935345367088,
  0.43381498090018267,
  0.4393414211438475],
 [1.4507318514034755,
  1.4220440561271417,
  0.4334828101644245,
  0.4358752166377816],
 [1.4427953943596168,
  1.4465196378516159,
  0.4356419199468527,
  0.43197573656845756],
 [1.431934640962328,
  1.4025770446456698,
  0.4384653712007972,
  0.4545060658578856],
 [1.4304970817788194,
  1.407234701374788,
  0.443614017605049,
  0.44974003466204504],
 [1.4277837700881562,
  1.4056939095310363,
  0.4451641477052538,
  0.45233968804159447],
 [1.4224618086814615,
  1.3952166910188037,
  0.44483197696949567,
  0.4510398613518198],
 [1.41743039024

In [10]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/Fer-2013/processed/landmarks/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1ferlandmarks.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.6830, Acurácia: 31.64%, Validação: Erro: 1.6209, Acurácia: 33.30%, Tempo: 234.67s
Validation accuracy improved to 0.3330. Saving the model.
Modelo salvo com sucesso em models\modelo1ferlandmarks.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.6225, Acurácia: 35.64%, Validação: Erro: 1.5989, Acurácia: 38.16%, Tempo: 143.68s
Validation accuracy improved to 0.3816. Saving the model.
Modelo salvo com sucesso em models\modelo1ferlandmarks.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.5921, Acurácia: 36.88%, Validação: Erro: 1.5815, Acurácia: 38.24%, Tempo: 143.99s
Validation accuracy improved to 0.3824. Saving the model.
Modelo salvo com sucesso em models\modelo1ferlandmarks.pt


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao..

[[1.6829784414218205,
  1.6209291142785431,
  0.31639262580966615,
  0.33304460805543523],
 [1.622500482610259,
  1.598908210843608,
  0.35641919946852685,
  0.3815504547423127],
 [1.5921079672261316,
  1.581521112715439,
  0.3687648784808725,
  0.38241663057600694],
 [1.5822727251823694,
  1.540044544888245,
  0.37684769971765486,
  0.4010394110004331],
 [1.571047484257659,
  1.528674934270734,
  0.3811105574932182,
  0.3993070593330446],
 [1.5626544717709394,
  1.527316695015479,
  0.38587167137241873,
  0.40666955391944565],
 [1.55212041096639,
  1.5211984205679647,
  0.3888612079942424,
  0.42052836725855347],
 [1.5582404412654642,
  1.51962771008881,
  0.3872557161047445,
  0.3949761801645734],
 [1.5488765305249996,
  1.5069090006618904,
  0.38919337873000054,
  0.4161974880900823],
 [1.5400282653774697,
  1.4984173326587304,
  0.39323478934839173,
  0.4213945430922477],
 [1.5381142914522885,
  1.5205597320951492,
  0.3950617283950617,
  0.41316587267215243],
 [1.526419312568662,


In [11]:

import sys
import os
from torchvision import transforms
import torch

# Caminho do dataset
dataset_path = '../data/Fer-2013/processed/landmarksFullConected/'

# Definir dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

# Transformações personalizadas
transformacoes_personalizadas = {
    'treino': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'validacao': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
}


# Preparar DataLoaders
data_loader_setup = data_loader_setup = DataLoaderSetup(dataset_path,image_size=224,batch_size=32,transformacoes=transformacoes_personalizadas)
data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# Carregar o modelo
model = ResnetV1(num_classes=num_classes)
model = model.to(device)
# model = CustomResNet50(num_classes).get_model().to(device)

# Nome do modelo salvo e paciência para early stopping
nameModel = 'modelo1ferconected.pt'
patience = 5

# Treinar e validar
trainer = Trainer(model, data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, device, num_classes, patience, nameModel)
epocas = 30
trainer.treinar_e_validar(epocas)


Dispositivo utilizado: cuda:0


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Época: 1/30

Executando a fase de treino...

Executando a fase de validacao...
Época 1/30, Treino: Erro: 1.5612, Acurácia: 38.61%, Validação: Erro: 1.4801, Acurácia: 41.58%, Tempo: 239.05s
Validation accuracy improved to 0.4158. Saving the model.
Modelo salvo com sucesso em models\modelo1ferconected.pt


Época: 2/30

Executando a fase de treino...

Executando a fase de validacao...
Época 2/30, Treino: Erro: 1.4477, Acurácia: 43.99%, Validação: Erro: 1.3918, Acurácia: 46.77%, Tempo: 147.31s
Validation accuracy improved to 0.4677. Saving the model.
Modelo salvo com sucesso em models\modelo1ferconected.pt


Época: 3/30

Executando a fase de treino...

Executando a fase de validacao...
Época 3/30, Treino: Erro: 1.4114, Acurácia: 45.57%, Validação: Erro: 1.3882, Acurácia: 46.77%, Tempo: 147.16s


Época: 4/30

Executando a fase de treino...

Executando a fase de validacao...
Época 4/30, Treino: Erro: 1.3907, Acurácia: 46.27%, Validação: Erro: 1.3667, Acurácia: 46.99%, Tempo: 147.38s
Valida

[[1.561186795191912,
  1.4801245764149582,
  0.3861144945188794,
  0.4157644001732352],
 [1.4476669507886786,
  1.391762332618882,
  0.43992913298638026,
  0.4677349501948896],
 [1.411447202944172,
  1.3882160154758154,
  0.4556527516332632,
  0.4677349501948896],
 [1.3907274816429849,
  1.3667448610906323,
  0.4626840881408482,
  0.4699003897791252],
 [1.3835883291044078,
  1.3580293085421251,
  0.4652308714428081,
  0.4794283239497618],
 [1.3742220547492026,
  1.3383855115494825,
  0.46849739785184363,
  0.48895625812039845],
 [1.365920659183941,
  1.3485952873362148,
  0.4730926807662496,
  0.4820268514508445],
 [1.3602752476854718,
  1.3307549075782945,
  0.4729819510574687,
  0.48679081853616285],
 [1.36302466271605, 1.342536185406255, 0.4733141401838113, 0.4833261152013859],
 [1.3499940318029566,
  1.323327268238654,
  0.4796810984387111,
  0.48722390645300995],
 [1.3451719602045855,
  1.3125341566365847,
  0.4754733695050382,
  0.4854915547856215]]

In [12]:
# import sys
# import os
# from torchvision import datasets, models, transforms
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Ensure the project root is in sys.path
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../../')))

# from src.data_processing.dataloader import DataLoaderSetup
# from src.models.resnet_V1 import ResnetV1
# from src.training.trainer import Trainer

# # Number of classes
# numero_de_classes = 8
# model = ResnetV1(num_classes=numero_de_classes)

# # Define loss function and optimizer
# funcao_erro = nn.NLLLoss()
# otimizador = optim.Adam(model.parameters(), lr=0.0001)

# # Dataset paths
# dataset = r'F:\Git\Teste\FER\affectnet\affectnet2'

# nameModel = 'affectnet.pt'

# # Custom transformations
# transformacoes_personalizadas = {
#     'treino': transforms.Compose([
#         transforms.RandomRotation(15),
#         transforms.Resize(256),
#         transforms.ToTensor()
#     ]),
#     'validacao': transforms.Compose([
#         transforms.Resize(256),
#         transforms.ToTensor()
#     ])
# }

# # Initialize DataLoaderSetup with custom transformations
# data_loader_setup = DataLoaderSetup(
#     dataset_path=dataset,
#     image_size=224,
#     batch_size=32,
#     transformacoes=transformacoes_personalizadas
# )

# # Get data loaders
# data_loader_treino, data_loader_validacao, num_imagens_treino, num_imagens_validacao, num_classes = data_loader_setup.get_data_loaders()

# # Verify data types
# data_iter = iter(data_loader_treino)
# images, labels = next(data_iter)
# print(f"Type of images: {type(images)}")   # Should be <class 'torch.Tensor'>
# print(f"Type of labels: {type(labels)}")   # Should be <class 'torch.Tensor'>

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Dispositivo utilizado: {device}")

# # Move the model to the device
# model = model.to(device)

# # Optionally, verify that the model is on the correct device
# for param in model.parameters():
#     print(f"Model parameter device: {param.device}")
#     break  # Check only the first parameter

# # Train and validate
# trainer = Trainer(
#     model=model,
#     data_loader_treino=data_loader_treino,
#     data_loader_validacao=data_loader_validacao,
#     num_imagens_treino=num_imagens_treino,
#     num_imagens_validacao=num_imagens_validacao,
#     device=device,
#     num_classes=num_classes,
#     name_model=nameModel,
#     otimizador=otimizador,
#     funcao_erro=funcao_erro
# )

# epocas = 30
# trainer.treinar_e_validar(epocas)